### Jul AdventKalender D15

https://adventofcode.com/2022/day/15

In [1]:
import numpy as np

#### Day 15.1 

There are a few sensors that can detect the **closest** beacon (measured by the **Manhattan distance**). There is never a tie where two beacons are the same distance to a sensor. Given the output of the sensors, for example:

    Sensor at x=2, y=18: closest beacon is at x=-2, y=15
    Sensor at x=9, y=16: closest beacon is at x=10, y=16
    Sensor at x=13, y=2: closest beacon is at x=15, y=3
    ......

Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor.

In the row where y=2000000, how many positions cannot contain a beacon?

In [2]:
def _parsePos(item):
    pos_x = item[(item.find('x=')+2) : (item.find(','))]
    pos_y = item[(item.find('y=')+2) :]
    return (int(pos_x), int(pos_y))

def readOutput(file_name):
    data_sensors = {}
    f = open(file_name, "r")
    while True:
        line = f.readline()
        if not line:
            break
        s_b = line.strip().split(':')
        data_sensors[_parsePos(s_b[0])] = [_parsePos(s_b[1])]
    f.close()
    return data_sensors

In [3]:
def mhDistance(pos_s, pos_b):
    return sum(abs(a-b) for a,b in zip(pos_s, pos_b))

In [4]:
def getMapWidth(data_sensors):
    width = 0
    start = 0
    for s,b in data_sensors.items():
        start_tmp = min(s[0],b[0][0])
        width = max(max(s[0],b[0][0]) - start, width)
        if start_tmp < start:
            start = start_tmp
            width -= (start-start_tmp)
    return width, start

In [5]:
def countNoneB(pos_y, map_width, map_left, data_sensors):
    res_count = 0
    for w in range(map_width+1):
        pos_x = w + map_left
        is_not_detected = True
        for s,bd in data_sensors.items():
            if mhDistance(s, (pos_x,pos_y)) <= bd[1] and (pos_x,pos_y)!=bd[0]:
                is_not_detected = False
                break
        if not is_not_detected:
            res_count += 1
    return res_count

In [6]:
# read data file to a dict with {sensor position: [beacon position, manhattan distance]}
data_sensors = readOutput("data/input15.txt")

# calculate and add distance between sensor and beacon to each item in the list
for s,bd in data_sensors.items():
    data_sensors[s].append(mhDistance(s,bd[0]))

# estimate the map size
map_width, map_left = getMapWidth(data_sensors)

y = 2000000 # target y
extend_width = 2000000 # estimated extend width of the map

# count the number of positions that are not possible beason positions at the target row y
countNoneB(y, map_width+extend_width, map_left-extend_width/2, data_sensors)

5878678

#### Day 15.2

Find one distress beacon which is not detected by any sensor. The distress beacon must have x and y coordinates each no lower than 0 and no larger than 4000000.

What is its tuning frequency, which can be found by multiplying its x coordinate by 4000000 and then adding its y coordinate.

In [7]:
# Now it is definitely not working to iterate every point like part 1 (where I can be a bit lazy)
# It is way too slow to check 4000000 x 4000000 points, and it is given that there is ONLY ONE target point
# So the ONE target point can only locate on the points where two border lines INTERSECT
# In fact, it can only locate on the intersections where FOUR lines intersect
# the border lines are the closest positions that can NOT be detected by the sensors

# find the boundaries for each sensor by using y = x + b (45 degree); y = -x + b (-45 degree)
# with considering the distance which each sensor can detect: y = (x +/- d) + b; y = -(x +/- d) + b
# then the bias terms: b = y - (x +/- d); b = y + (x +/- d)
def findBoundaries(data_sensors):
    bs_p, bs_n = [], []
    for s,bd in data_sensors.items():
        bs_p.append(s[1]-s[0]+(bd[1]+1))
        bs_p.append(s[1]-s[0]-(bd[1]+1))
        bs_n.append(s[0]+s[1]+(bd[1]+1))
        bs_n.append(s[0]+s[1]-(bd[1]+1))
    bs_p = [b for b in bs_p if bs_p.count(b) >= 2]
    bs_n = [b for b in bs_n if bs_n.count(b) >= 2]
    return bs_p, bs_n

# the intersect points between the 45 degree boundries and the -45 degress boundaries are:
# y = x + b_p;
# y = -x + b_n
# 2y = b_p + b_n => y = (b_p + b_n)/2
# 0 = 2x + b_p - b_n => x = (b_n - b_p)/2
# with the shape of the region that all the sensors covers
def findPossibleB(search_size, data_sensors, bs_p, bs_n):
    for b_p in bs_p:
        for b_n in bs_n:
            b_inter = ((b_n - b_p)//2, (b_p + b_n)//2)
            # if intersection is in the search area
            if all(search_size[0] < x_y < search_size[1] for x_y in b_inter):
                # further check if it fulfill the condition that it was not detected by any sensors
                is_detected = False
                for s,bd in data_sensors.items():
                    if mhDistance(b_inter, s) <= bd[1]:
                        is_detected = True
                        break
                if not is_detected:
                    return b_inter

In [8]:
data_sensors = readOutput("data/input15.txt")

for s,bd in data_sensors.items():
    data_sensors[s].append(mhDistance(s,bd[0]))
    
bs_p, bs_n = findBoundaries(data_sensors)

search_size = (0,4000000) # search size for both x and y
pos_target_b = findPossibleB(search_size, data_sensors, bs_p, bs_n)
pos_target_b[0] * 4000000 + pos_target_b[1]

11796491041245